In [4]:
import sys

print(sys.path)

sys.path.append('/home/mishu/coding/CriminalTracker/lib/python3.11')
import pandas as pd
from geopy.geocoders import Nominatim
import asyncio
import threading
import pickle
import sqlite3
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from playwright.async_api import Page, expect, async_playwright, Playwright, TimeoutError
import random
import numpy as np
from datetime import datetime
#from homeharvest import scrape_property
#from sqlalchemy import create_engine
import requests
import matplotlib.pyplot as plt
from homeharvest import scrape_property
from datetime import datetime

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\python310.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\chuch\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0', 'c:\\Users\\chuch\\OneDrive\\Desktop\\coding\\CriminalTracker\\.venv', '', 'c:\\Users\\chuch\\OneDrive\\Desktop\\coding\\CriminalTracker\\.venv\\lib\\site-packages', 'c:\\Users\\chuch\\OneDrive\\Desktop\\coding\\CriminalTracker\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\chuch\\OneDrive\\Desktop\\coding\\CriminalTracker\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\chuch\\OneDrive\\Desktop\\coding\\CriminalTracker\\.venv\\lib\\site-packages\\Pythonwin']


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy.optimize import curve_fit

In [6]:
from sqlalchemy import create_engine, types

In [ ]:
states = [ 
"NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]


In [ ]:
conn = sqlite3.connect('HouseData.db')
cursor = conn.cursor()


In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';")
names = cursor.fetchall()    
print(names)

In [ ]:
def propertiesTosSql(properties,state):
    properties.to_sql(state, conn, if_exists='replace', index=False)
    cursor.execute("SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';")
    databaseNames = cursor.fetchall()
    print(databaseNames)
    print(cursor.fetchall())

In [ ]:
for state in states:
  

    print("this is the state we are on: " + state)
    current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"HomeHarvest_{state}.csv"
    properties = scrape_property(
    location= state,
    listing_type="for_sale",  # or (for_sale, for_rent, pending)
  # sold in last 30 days - listed in last 30 days if (for_sale, for_rent)

  # date_from="2023-05-01", # alternative to past_days
  # date_to="2023-05-28",
  # foreclosure=True
  # mls_only=True,  # only fetch MLS listings
)
    print(len(properties))
    
    if 'agent_phones' in properties.columns:
      properties.drop('agent_phones', axis = 1, inplace=True)
    if (len(properties) > 1000):
      propertiesTosSql(properties.sample(n=1000),state)
    else:
      propertiesTosSql(properties,state)
  



In [ ]:
cursor.execute("select state from NY ")
rows = cursor.fetchall()
print(len(rows))

In [ ]:






query  = """

CREATE TABLE StateHouseData2 AS
SELECT * FROM AL
UNION
SELECT * FROM AK
UNION
SELECT * FROM AZ
UNION
SELECT * FROM AR
UNION
SELECT * FROM CA
UNION
SELECT * FROM CO
UNION
SELECT * FROM CT
UNION
SELECT * FROM DE
UNION
SELECT * FROM FL
UNION
SELECT * FROM GA
UNION
SELECT * FROM HI
UNION
SELECT * FROM ID
UNION
SELECT * FROM IL
UNION
SELECT * FROM "IN"
UNION
SELECT * FROM IA
UNION
SELECT * FROM KS
UNION
SELECT * FROM KY
UNION
SELECT * FROM LA
UNION
SELECT * FROM ME
UNION
SELECT * FROM MD
UNION
SELECT * FROM MA
UNION
SELECT * FROM MN
UNION
SELECT * FROM MS
UNION
SELECT * FROM MO
UNION
SELECT * FROM MT
UNION
SELECT * FROM NE
UNION
SELECT * FROM NV
UNION
SELECT * FROM NH
UNION
SELECT * FROM NM
UNION
SELECT * FROM NY
UNION
SELECT * FROM NC
UNION
SELECT * FROM ND
UNION
SELECT * FROM OH
UNION
SELECT * FROM OK
UNION
SELECT * FROM "OR"
UNION
SELECT * FROM PA
UNION
SELECT * FROM RI
UNION
SELECT * FROM SC
UNION
SELECT * FROM SD
UNION
SELECT * FROM TN
UNION
SELECT * FROM TX
UNION
SELECT * FROM UT
UNION
SELECT * FROM VT
UNION
SELECT * FROM VA
UNION
SELECT * FROM WA
UNION
SELECT * FROM WV
UNION
SELECT * FROM WI
UNION
SELECT * FROM WY
"""

cursor.execute(query)

In [ ]:
def getCensusSquareMiles():
    print("IN READ CENSUS")
    conn = sqlite3.connect("HouseData.db")
    cursor = conn.cursor()

    df = pd.read_csv('2022_Gaz_116CDs_national.txt', sep='\t')
    

   
    
    cursor.execute("SELECT GEOID, County FROM censusData")
    


    
    extracteddata = df[['USPS', 'GEOID', 'NAME', 'ALAND_SQMI']]
    extracteddata = extracteddata.rename( columns= {"USPS" : "state", "GEOID": "code", "NAME" : "County" , 'ALAND' : 'area'})
    print(extracteddata)
    extracteddata.to_sql("censusData", conn, if_exists='replace', index=False)


In [ ]:
def getCountyCode(state, checkCounty):
    print("this is the county we are workign with:" + checkCounty)
    cursor.execute ( "SELECT code FROM checkCensusReal WHERE state = ? AND name = ?", (state, checkCounty))
    
    try:
        countyCode = str(cursor.fetchall()[0])
        
        
        
        countyCode = countyCode.replace("(", "")
        countyCode = countyCode.replace(")", "")
        countyCode = countyCode.replace(",", "")
        print("got the county code:" + str(countyCode) )
        print(type(countyCode))
        countyCodeInsert = countyCode[-3:]
        print("this is the countyCodeInsert:" + countyCodeInsert)
        
        
        return countyCodeInsert
    except IndexError:
        return None

In [ ]:
#here we are going to get the second countycode2
def getCountyCode2(state, checkCounty):
    print("this is the county we are workign with:" + checkCounty)
    cursor.execute ( "SELECT code FROM checkCensusReal WHERE state = ? AND county = ?", (state, checkCounty))
    
    try:
        countyCode = str(cursor.fetchall()[0])
        
        
        
        countyCode = countyCode.replace("(", "")
        countyCode = countyCode.replace(")", "")
        countyCode = countyCode.replace(",", "")
        print("got the county code:" + str(countyCode) )
        print(type(countyCode))
        countyCodeInsert = countyCode[-4:]
        print("this is the countyCodeInsert:" + countyCodeInsert)
        
        
        return countyCodeInsert
    except IndexError:
        return None

In [ ]:


def getCountiesAndDistricts():
    file_path = 'countiesAndDistricts.txt'

# Define column specifications and names
    col_specs = [(0, 2), (5, 7), (10, 13), (16, -1)]
    col_names = ['State', 'District', 'CountyCode', 'Name and History']

    # Read the data using fixed-width column specification
    df = pd.read_fwf(file_path, colspecs=col_specs, header=None)
    df.columns = col_names

    # Optionally, you might want to split 'Name and History' into separate 'Name' and 'History Flag' columns
    df[['Name', 'HistoryFlag']] = df['Name and History'].str.rsplit(n=1, expand=True)

    df['Name'] = df['Name'].str.replace('\t|\d', '', regex=True)
    
    df.to_sql("CountiesAndDistricts", conn, if_exists='replace', index=False)



In [ ]:
getCountiesAndDistricts()

In [ ]:
def getArea():
    df = pd.read_csv("2020_Gaz_116CDs_national.txt", sep = '\t')

    print(df.columns)


    df['USPS'] = df['USPS'].astype(str)
    df['GEOID'] = df['GEOID'].astype(str)
    df['ALAND'] = df['ALAND'].astype(float)
    df['AWATER'] = df['AWATER'].astype(float)
    df['ALAND_SQMI'] = df['ALAND_SQMI'].astype(float)
    df['AWATER_SQMI'] = df['AWATER_SQMI'].astype(float)
    df['INTPTLAT'] = df['INTPTLAT'].astype(float)
    
    print(df.head())
    df.to_sql("areaCensus", conn, if_exists='replace', index='false')


In [ ]:
getArea()

In [ ]:

cursor.execute("PRAGMA table_info('areaCensus')")
columns = cursor.fetchall()

# Find and print the type of the specific column
for column in columns:
    if column[1] == 'GEOID':
        print(f"Column Type for '{column[1]}': {column[2]}")

print(cursor.fetchall())

In [17]:
def getCountyCode(state, checkCounty):
    stateCode = state_to_fips(state)
    print("this is the stateCode: " + stateCode)
   # print("this is the county we are workign with:" + checkCounty)
    



    cursor.execute ( "SELECT State,District,CountyCode FROM CountiesAndDistricts WHERE state = ? AND Name = ?", (state_to_fips(state), checkCounty))
    
    try:
        (stateCode, DistrictCode,countyCode)= cursor.fetchall()[0]
        
       


        finalDistrictCode = DistrictCode[1] + DistrictCode[0]

       # print("this is the districtCode: " + finalDistrictCode)
        geoid = (stateCode + finalDistrictCode)

        #print(type(geoid))
        cursor.execute("SELECT ALAND_SQMI,  AWATER_SQMI FROM areaCensus WHERE GEOID = ?", (geoid,))
        
        
        area = cursor.fetchall()

        if (area):
            print(str(area))
            area1 = area[0][0]
            area2 = area[0][1]
            totalArea = area1+area2
            print("this is the area: " + str(totalArea))
    
        
            return ( (finalDistrictCode),totalArea)
        else:
            print("NO AREA WAS FOUND FOR COUNTY: " + checkCounty + "FOR CODE:" + geoid + "with disrict Number: "+ str(finalDistrictCode) + "and state code " + str(stateCode)) 

            return None
    
    except IndexError:
        return None

In [18]:
def getArea(state, checkCounty):
    cursor.execute ( "SELECT area FROM countiesAndDistricts WHERE state = ? AND district = ?", (state, checkCounty))
    area = str(cursor.fetchall()[0])
    area = area.replace("(", "")
    area = area.replace(")", "")
    area = area.replace(",", "")
    return int(area)

In [ ]:
cursor.execute("SELECT density from stateHouseData WHERE state = ? ", ("CA",))
densities = cursor.fetchall()
print(len(densities))

In [19]:
def state_to_fips(state_input):
    # Mapping of state names and abbreviations to FIPS codes
    states = {
        "Alabama": "01", "AL": "01",
        "Alaska": "02", "AK": "02",
        "Arizona": "04", "AZ": "04",
        "Arkansas": "05", "AR": "05",
        "California": "06", "CA": "06",
        "Colorado": "08", "CO": "08",
        "Connecticut": "09", "CT": "09",
        "Delaware": "10", "DE": "10",
        "Florida": "12", "FL": "12",
        "Georgia": "13", "GA": "13",
        "Hawaii": "15", "HI": "15",
        "Idaho": "16", "ID": "16",
        "Illinois": "17", "IL": "17",
        "Indiana": "18", "IN": "18",
        "Iowa": "19", "IA": "19",
        "Kansas": "20", "KS": "20",
        "Kentucky": "21", "KY": "21",
        "Louisiana": "22", "LA": "22",
        "Maine": "23", "ME": "23",
        "Maryland": "24", "MD": "24",
        "Massachusetts": "25", "MA": "25",
        "Michigan": "26", "MI": "26",
        "Minnesota": "27", "MN": "27",
        "Mississippi": "28", "MS": "28",
        "Missouri": "29", "MO": "29",
        "Montana": "30", "MT": "30",
        "Nebraska": "31", "NE": "31",
        "Nevada": "32", "NV": "32",
        "New Hampshire": "33", "NH": "33",
        "New Jersey": "34", "NJ": "34",
        "New Mexico": "35", "NM": "35",
        "New York": "36", "NY": "36",
        "North Carolina": "37", "NC": "37",
        "North Dakota": "38", "ND": "38",
        "Ohio": "39", "OH": "39",
        "Oklahoma": "40", "OK": "40",
        "Oregon": "41", "OR": "41",
        "Pennsylvania": "42", "PA": "42",
        "Rhode Island": "44", "RI": "44",
        "South Carolina": "45", "SC": "45",
        "South Dakota": "46", "SD": "46",
        "Tennessee": "47", "TN": "47",
        "Texas": "48", "TX": "48",
        "Utah": "49", "UT": "49",
        "Vermont": "50", "VT": "50",
        "Virginia": "51", "VA": "51",
        "Washington": "53", "WA": "53",
        "West Virginia": "54", "WV": "54",
        "Wisconsin": "55", "WI": "55",
        "Wyoming": "56", "WY": "56"
    }

    # Convert input to uppercase for flexible matching
    state_input_upper = state_input.upper()

    # Find and return the FIPS code
    fips_code = states.get(state_input_upper, "State not found")
    return fips_code



In [ ]:
cursor.execute("ALTER TABLE stateHouseData ADD DENSITY FLOAT")

In [ ]:
 'key': 'ba62c3e29b0e31f487851f8b8543afa2fd271e3e'

In [20]:
us_census_key = ""


cursor.execute("SELECT DISTINCT State, county FROM StateHouseData")
stateData = cursor.fetchall()
#cursor = conn.cursor()
#cursor.execute("SELECT DISTINCT State, county FROM StateHouseData")
#rows = cursor.fetchall()

# Construct the base URL for the API request
#url = "https://api.census.gov/data/2021/pep/population"
#url = "https://api.census.gov/data/2021/pep/population?get=DENSITY_2021,POP_2021,NAME&for=state:06&key={us_census_key}"

for (state, county) in stateData:
    url = "https://api.census.gov/data/2020/acs/acs5"
    
    if (state != None and county != None):
        print("this is the state:" + state)
        print("this is the county: " + county)
        
        tupRet = getCountyCode(state, county)

        
        if (tupRet):
            (districtCode, area) = tupRet

            print("this is the districtCode: " + str(districtCode))
            #print("this is the area: " + str(area))
            fipState = state_to_fips(state)
            print("this is the stateCode: " + str(fipState))
            params = {
            'get': 'B01003_001E',  # Total population
            'for': f'congressional district:{districtCode}',
            'in': f'state:{fipState}',
            'key': ''
             }
            url = "https://api.census.gov/data/2020/acs/acs5"
            
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                print("this is the data:" + str(data))
                population = data[1][0]
                print("this is the county:" + county)
                density = float(float(population)/area)
                print("DENSITY: " + str(density))
                cursor.execute("UPDATE stateHouseData SET Density = ? WHERE county = ? AND state = ?", ( (float(population)/area), county, state))    

            else:
                print("Failed to retrieve data:", response.status_code)
                print("Response text:", response.text)


# Set the parameters for the request
#params = {
#    'get': 'DENSITY_2021,POP_2021,Orange County,California',  # The data fields you want to retrieve
#    'for': 'region:*',  # Fetch data for all regions
 #   'key': us_census_key


this is the state:OR
this is the county: Multnomah
this is the stateCode: 41
[(3007.117, 297.087)]
this is the area: 3304.204
this is the districtCode: 01
this is the stateCode: 41
this is the data:[['B01003_001E', 'state', 'congressional district'], ['851961', '41', '01']]
this is the county:Multnomah
DENSITY: 257.8415255232425
this is the state:WA
this is the county: Clark
this is the stateCode: 53
[(6181.857, 423.384)]
this is the area: 6605.241
this is the districtCode: 01
this is the stateCode: 53
this is the data:[['B01003_001E', 'state', 'congressional district'], ['768331', '53', '01']]
this is the county:Clark
DENSITY: 116.32141809814358
this is the state:NV
this is the county: Clark
this is the stateCode: 32
NO AREA WAS FOUND FOR COUNTY: ClarkFOR CODE:3208with disrict Number: 08and state code 32
this is the state:CA
this is the county: Butte
this is the stateCode: 06
[(1730.603, 123.219)]
this is the area: 1853.8220000000001
this is the districtCode: 05
this is the stateCode:

In [ ]:
def deleteDuplicate():
    query = """DELETE FROM stateHouseData
    WHERE rowid NOT IN (
    SELECT rowid
    FROM (
    SELECT rowid, RANK() OVER (PARTITION BY column1, column2 ORDER BY date_column DESC) as rank
    FROM stateHouseData
  ) WHERE rank = 1
    );b"""
    cursor.execute(query)

In [ ]:
deleteDuplicate()

In [ ]:
cursor.execute("PRAGMA table_info(StateHouseData)")
columns = cursor.fetchall()
print(columns)
rows = (cursor.fetchall())
print(len(rows))
# Print the name of each column


In [16]:
us_census_key = ""


cursor.execute("SELECT DISTINCT State, county FROM StateHouseData")
stateData = cursor.fetchall()
#cursor = conn.cursor()
#cursor.execute("SELECT DISTINCT State, county FROM StateHouseData")
#rows = cursor.fetchall()

# Construct the base URL for the API request
#url = "https://api.census.gov/data/2021/pep/population"
#url = "https://api.census.gov/data/2021/pep/population?get=DENSITY_2021,POP_2021,NAME&for=state:06&key={us_census_key}"

for (state, county) in stateData:
    url = "https://api.census.gov/data/2020/acs/acs5"
    
    if (state != None and county != None):
        print("this is the state:" + state)
        print("this is the county: " + county)
        
        tupRet = getCountyCode(state, county)
        if (tupRet):
            (countyCode, area) = tupRet

            print("this is the countyCode: " + str(countyCode))
            print("this is the area: " + str(area))
            fipState = state_to_fips(state)
            print("this is the districtCode: " + str(countyCode))
            params = {
            'get': 'B01003_001E',  # Total population
            'for': f'congressional district:{countyCode}',
            'in': f'state:{fipState}',
            'key': 'your_actual_api_key_here'
             }
            url = "https://api.census.gov/data/2020/acs/acs5"
            
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                print("this is the data:" + str(data))
                population = data[1][0]
                print("this is the county:" + county)
                density = float(float(population)/area)
                print("DENSITY: " + str(density))
                cursor.execute("UPDATE StateHouseData SET Density = ? WHERE county = ? AND state = ?", ( (float(population)/area), county, state))    

            else:
                print("Failed to retrieve data:", response.status_code)
                print("Response text:", response.text)


# Set the parameters for the request
#params = {
#    'get': 'DENSITY_2021,POP_2021,Orange County,California',  # The data fields you want to retrieve
#    'for': 'region:*',  # Fetch data for all regions
 #   'key': us_census_key


this is the state:OR
this is the county: Multnomah


NameError: name 'getCountyCode' is not defined

In [ ]:
def getCriminals():
     cursor.execute("SELECT * FROM CA ", state)

In [ ]:
cursor.execute("ALTER TABLE stateHouseData ADD DENSITY FLOAT")

In [ ]:
cursor.execute("ALTER TABLE StateHouseData ADD  numCriminals INT")

In [ ]:
def deleteDup():
    query = """ DELETE FROM StateHouseData
    WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM StateHouseData
    GROUP BY property_url
    );"""
    cursor.execute(query)

In [ ]:
deleteDup()

In [ ]:
def getCriminals():

    
   
    cursor.execute("SELECT property_url, Density, latitude, longitude FROM StateHouseData WHERE state = ?", ("CA",))
    retrieved  = cursor.fetchall()
    print(len(retrieved))
        
    i = 0
    for retrieve in retrieved:
        if (i < 1200):
                property_url = retrieve[0]
                density = retrieve[1]
                latitude = retrieve[2]
                longitude = retrieve[3]
                
                cursor.execute("SELECT numCriminals FROM StateHouseData Where property_url = ?", (property_url,))
                numCriminalsCheck = cursor.fetchall()
                print(len(numCriminalsCheck))
                if (len(numCriminalsCheck) == 0):
       
                        print("this is the density: " + str(density))
                        if (density != None):
                                density = float(density)
                                if (density > 1000):
                                        print("IN DENSE AREA")
                                        print("Found dense area")
                                        radius = '1mi'
                                elif (density >= 750 and density < 1000):
                                        radius = '1.5mi'
                                elif (density >= 500 and density < 750):
                                        radius='2mi'
                                elif (density >= 250 and density < 500):
                                        radius='3mi'
                                elif (density ):
                                        radius = '5mi'
                                        
                                        
                                url= f'https://api.crimeometer.com/v3/sex-offenders/location?lat={latitude}&lon={longitude}&radius={radius}'
                                headers = {
                                        'Content-Type': 'application/json',
                                        'x-api-key':  'TEGWymWxYw1pP53wdeHk2tFufItU7Cb1wrL02nX4'
                                        }
                                response = requests.get(url, headers=headers)
                                print(response)
                                if (response.status_code == 200):
                                        data = response.json()
                                        print(data)
                                        total_criminals = data['sex_offenders_count']
                                        print("Total criminals:", total_criminals)
                                        cursor.execute("UPDATE StateHouseData  SET numCriminals = ? WHERE property_url = ?", (total_criminals,property_url))
                                        cursor.execute("SELECT property_url, numCriminals FROM StateHouseData WHERE property_url = ?", (property_url,))
                                        rows = cursor.fetchall()
                                        print(rows)
                                else:
                                        print("criminalsNotFound")
                                i+=1
            

In [ ]:
def foop():
    print("foop")

In [15]:
cursor.execute("SELECT * from StateHouseData where state = ?" , ("CA",))
densities = cursor.fetchall()[0]
print(densities)

('https://www.realtor.com/realestateandhomes-detail/1001247236', 'MRCA', 'SN24085657', 'FOR_SALE', 'A TRUE one-of-a-kind estate nestled on a picturesque 21.44-acre almond orchard minutes from both Chico and Durham. This spectacular retreat offers an unparalleled blend of luxury, privacy and functionality. Boasting over 7, 000 square feet of living space, the residence features 5 bedrooms and 6 bathrooms, complemented by inviting formal living and dining areas. The gourmet kitchen, complete with double ovens, dual sinks and dishwashers is a culinary enthusiasts and entertainers dream. Social and family options abound with a media room featuring surround sound, a separate billiard loft, and an exercise room. Ascend the 4-story lookout tower equipped as a poker room with wet bar, and youll find breathtaking valley views over your orchard. An 1, 100-bottle wine cellar adds a touch of elegance for the collector. The outdoor oasis includes a heated in-ground pool, fire pit, and expansive cov

In [27]:
def getCriminals():

    
   
       cursor.execute("SELECT property_url, Density, latitude, longitude FROM StateHouseData WHERE state = ?", ("CA",))
       retrieved  = cursor.fetchall()
       print(len(retrieved))
       i = 0
       for i,retrieve in enumerate(retrieved):
              if (i < 500):
                     property_url = retrieve[0]
                     density = retrieve[1]
                     latitude = retrieve[2]
                     longitude = retrieve[3]
                
               
       
              print("this is the density: " + str(density))
                  
              radius = '3mi'
                                        
                                        
              url= f'https://api.crimeometer.com/v3/sex-offenders/location?lat={latitude}&lon={longitude}&radius={radius}'
              headers = {
                     'Content-Type': 'application/json',
                     'x-api-key':  'TEGWymWxYw1pP53wdeHk2tFufItU7Cb1wrL02nX4'
              }
              response = requests.get(url, headers=headers)
              print(response)
              if (response.status_code == 200):
                     data = response.json()
                     print(data)
                     total_criminals = data['sex_offenders_count']
                     print("Total criminals:", total_criminals)
                     cursor.execute("UPDATE StateHouseData  SET numCriminals = ? WHERE property_url = ?", (total_criminals,property_url))
                     cursor.execute("SELECT property_url, numCriminals FROM StateHouseData WHERE property_url = ?", (property_url,))
                     rows = cursor.fetchall()
                     print(rows)
                        
                     i+=1
                     if (i > 500):
                         break
                      
            

In [28]:
getCriminals()

1985
this is the density: 391.07530280684983
<Response [200]>
{'sex_offenders_count': 195, 'pages_count': 2, 'sex_offenders': [{'sex_offender_state': 'CA', 'sex_offender_location': 'CHICO', 'sex_offender_zipcode': '95926', 'sex_offender_lat': 39.74239, 'sex_offender_lon': -121.84645, 'sex_offender_name': 'CANDALARIA, JOHN E', 'sex_offender_first_name': 'JOHN', 'sex_offender_middle_name': 'E', 'sex_offender_last_name': 'CANDALARIA', 'sex_offender_birthdate': '1946-09-20', 'sex_offender_age': 77, 'sex_offender_address_line1': '1436 OLEANDER AVE', 'sex_offender_address_line2': 'CHICO,CA 95926', 'sex_offender_sex': 'MALE', 'sex_offender_race': 'WHITE', 'sex_offender_height': '5ft. 05in.', 'sex_offender_weight': '140', 'sex_offender_hair': 'GRAY/PARTIALLY GRAY', 'sex_offender_eyes': 'BROWN', 'sex_offender_marks': 'ANKLE, LEFTUNKNOWN;ANKLE, RIGHTUNKNOWN;BACK;FINGER(S), LEFT HAND;HAND, LEFTWORD;TATTOO, UNKNOWNINITIAL;TATTOO, UNKNOWNWORD', 'sex_offender_aliases': 'JOHN EDDY CANDALARIA;JOHN EDD

In [7]:
conn = sqlite3.connect("HouseData.db")
cursor = conn.cursor()

In [8]:


cursor.execute("SELECT numCriminals FROM StateHouseData WHERE state= 'CA'")
numberOfCriminals = cursor.fetchall()
print(numberOfCriminals[0:500])


[(None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), 

In [ ]:
def getCriminals2():

    
   
    cursor.execute("SELECT property_url, Density, latitude, longitude FROM StateHouseData2 WHERE state = ?", ("CA",))
    retrieved  = cursor.fetchall()
    print(len(retrieved))
        
    i = 0
    for retrieve in retrieved:
        if (i < 500):
                property_url = retrieve[0]
                density = retrieve[1]
                latitude = retrieve[2]
                longitude = retrieve[3]
                
               
       
                print("this is the density: " + str(density))
                  
                radius = '3mi'
                                        
                                        
                url= f'https://api.crimeometer.com/v3/sex-offenders/location?lat={latitude}&lon={longitude}&radius={radius}'
                headers = {
                            'Content-Type': 'application/json',
                            'x-api-key':  ''
                            }
                response = requests.get(url, headers=headers)
                print(response)
                if (response.status_code == 200):
                    data = response.json()
                    print(data)
                    total_criminals = data['sex_offenders_count']
                    print("Total criminals:", total_criminals)
                    cursor.execute("UPDATE StateHouseData2  SET numCriminals = ? WHERE property_url = ?", (total_criminals,property_url))
                    cursor.execute("SELECT property_url, numCriminals FROM StateHouseData2 WHERE property_url = ?", (property_url,))
                    rows = cursor.fetchall()
                    print(rows)
                        
                    i+=1
                    if (i > 500):
                         break
            

In [ ]:
getCriminals2()

In [ ]:
cursor.execute("PRAGMA table_info(stateHouseData)")
names = cursor.fetchall()
print(names)

cursor.execute("SELECT  Density FROM StateHouseData" )
rows = cursor.fetchall()
print(rows[3])

In [ ]:
getCriminals()

In [ ]:
def neg_exp_decay(x,a,b,c):
    return a * np.exp(-b * x) + c

In [ ]:
def predictSafety():




    # Fetching data
    cursor.execute("PRAGMA table_info(StateHouseData2)")
    names = cursor.fetchall()
    print(names)
    cursor.execute("SELECT list_price,numCriminals FROM StateHouseData2 WHERE state = ? AND list_price < 1000000", ("CA",))
    finalData = cursor.fetchall()

    x = []
    y = []
    for (price, numCriminals) in finalData:
        x.append(price)
        y.append(numCriminals)

    print(x)
    print(y)

    
    data = {'x': x, 'y': y}
    df = pd.DataFrame(data)

    plt.scatter(df['x'], df['y'], color='blue', label='Original Data')

    params, covariance = curve_fit(neg_exp_decay, df['x'], df['y'])
    df['predicted'] = neg_exp_decay(df['x'], *params)
    df['residuals'] = df['y'] - df['predicted']

# Calculate the z-score of the residuals
    median_residuals = np.median(df['residuals'])
    mad_residuals = np.median(np.abs(df['residuals'] - median_residuals))
    df['residuals_mad'] = np.abs(df['residuals'] - median_residuals) / mad_residuals

# Define a threshold for identifying outliers using MAD
    threshold = 3
    df_no_outliers = df[df['residuals_mad'] <= threshold]


# Re-fit the model to the data without outliers
    params_no_outliers, covariance_no_outliers = curve_fit(neg_exp_decay, df_no_outliers['x'], df_no_outliers['y'])
    df_no_outliers['predicted'] = neg_exp_decay(df_no_outliers['x'], *params_no_outliers)

# Plot the data
    plt.figure(figsize=(10, 6))

# Plot original data
    plt.scatter(df['x'], df['y'], color='blue', label='Original Data')
# Plot fitted curve on original data
    plt.plot(df['x'], df['predicted'], color='blue', linestyle='--', label='Fitted Curve (Original Data)')

# Plot data without outliers
    plt.scatter(df_no_outliers['x'], df_no_outliers['y'], color='red', label='Data without Outliers')
# Plot fitted curve on data without outliers
    plt.plot(df_no_outliers['x'], df_no_outliers['predicted'], color='red', label='Fitted Curve (No Outliers)')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Negative Exponential Decay Model with Outlier Removal')
    plt.legend()
    plt.show()

    return df

In [ ]:
df = predictSafety()

In [ ]:
plt.scatter(df['x'], df['y'], color='blue', label='Original Data')

In [ ]:
def normalizeData(hashy):
    addresses = []
    prices = []
    numCriminals = []
    for key in hashy.keys():
        #print("THIS IS THE KEY:" + str(key))

        #if there is a dollar sign, then we have the price and the number of criminals nearby
        #print("this is key[1]:" + str(key[1])
        
            #if ("$" in key[1]):
           #     price = key[1].replace('$', '')
          #      price = price.replace(',', '')
            
        #    if ("From" in key[1]):
         #       price = price.replace("From", "")
        
        #print("this is the price:" + str(price))
        price = key[1].replace(",", "")
        
        prices.append(price)
        addresses.append(key[0])
        
        if (hashy[key] == 0):
            numCriminals.append(0)
        else:
            numCriminals.append(hashy[key])

    print("length of addresses:" + str(len(addresses)))
    print("length of prices:" + str(len(prices)))
        


    
  
    

    numCriminals = np.array(numCriminals).reshape(-1, 1)
    numCriminals.remove(maxIndex)
    scaler_price = MinMaxScaler(feature_range=(0, 4))
    scaler_criminals =MinMaxScaler(feature_range=(0, 2))
    scaled_prices = scaler_price.fit_transform(prices)
    scaled_numCriminals = scaler_criminals.fit_transform(numCriminals)

    # Creating the DataFrame
    df = pd.DataFrame({
        'Price': scaled_prices.flatten(),  # Flatten to convert from 2D back to 1D
        'numCriminals': scaled_numCriminals.flatten()
    })

    # Calculate the safety rating
    df["safetyRating"] = df['numCriminals'] + df['Price'] / 2


    
    plt.scatter(df["Price"], df["numCriminals"])

    plt.show()
    
    
    return df

In [ ]:
cursor.execute('''
CREATE TABLE StateHouseData2 AS
SELECT *
FROM StateHouseData
WHERE 1=0;
''')

cursor.execute('''
INSERT INTO StateHouseData2
SELECT *
FROM StateHouseData;
''')

In [ ]:
cursor.execute("SELECT * from StateHouseData")
rows = cursor.fetchall()

print(rows[12])

